# import 

In [1]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_bounds
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import geopandas as gpd
from glob import glob
from scipy.stats import entropy, mode
from skimage.util.shape import view_as_blocks
import pyproj
import utm
from affine import Affine
from pyproj import Transformer
from shapely.geometry import Polygon, MultiPolygon
import cv2
from joblib import Parallel, delayed
import json

# functions

In [2]:
def load_topojson(topojson_path, src_crs=4326):
    # Load the TopoJSON file directly with GeoPandas
    gdf = gpd.read_file(topojson_path)
    
    # Manually set the CRS to EPSG:4326
    gdf.set_crs(epsg=src_crs, inplace=True)
    
    return gdf

def reproject_to_utm_and_calculate_area(gdf):
    ''' Use the centroid to calculate the UTM zone 
    '''
    # Temporary reprojection to an equal area projection to calculate the centroid
    gdf_equal_area = gdf.to_crs(epsg=6933) # EPSG:6933 is the World Equidistant Cylindrical projection
    centroid_equal_area = gdf_equal_area.geometry.union_all().centroid
    gdf_temp = gpd.GeoDataFrame(geometry=gpd.GeoSeries([centroid_equal_area]))
    gdf_temp.set_crs(epsg=6933, inplace=True)    
    gdf_temp = gdf_temp.to_crs(epsg=4326) # convert back to geographic coordinates 
    lon,lat = gdf_temp.geometry[0].coords[0]
    
    x = utm.from_latlon(lat, lon) # easting, northing, zone_number, band_letter
    
    def infer_utm_epsg(utm_tuple):
        easting, northing, zone_number, band_letter = utm_tuple
        # Determine the hemisphere from the band letter
        if 'N' <= band_letter <= 'X':  # Northern Hemisphere. Code starts with 326
            epsg_code = 32600 + zone_number 
        elif 'C' <= band_letter <= 'M':  # Southern Hemisphere. Code starts with 327
            epsg_code = 32700 + zone_number
        else:
            raise ValueError("Invalid band letter")
        return epsg_code

    epsg_code = infer_utm_epsg(x)
    
    # Reproject the original GeoDataFrame to the determined UTM zone
    gdf_utm = gdf.to_crs(epsg=epsg_code)

    # Calculate the area of the reprojected GeoDataFrame
    gdf_utm['area'] = gdf_utm.geometry.area

    # Calculate the total area
    total_area = gdf_utm['area'].sum()

    return epsg_code, total_area, (lat, lon)

def align_rasters(dem_path, land_cover_path, dst_crs, resolution=None):
    # calculate destination affine, width and height of dem and land cover in the destination coordiantes
    with rasterio.open(dem_path) as dem_src:
        dem_transform, dem_width, dem_height = calculate_default_transform(
            dem_src.crs, dst_crs, dem_src.width, dem_src.height, *dem_src.bounds)
        dem_bounds = transform_bounds(dem_src.crs, dst_crs, *dem_src.bounds)
        
    with rasterio.open(land_cover_path) as lc_src:
        lc_transform, lc_width, lc_height = calculate_default_transform(
            lc_src.crs, dst_crs, lc_src.width, lc_src.height, *lc_src.bounds)
        lc_bounds = transform_bounds(lc_src.crs, dst_crs, *lc_src.bounds)

    # Calculate intersection bounds
    intersection_bounds = (
        max(dem_bounds[0], lc_bounds[0]),
        max(dem_bounds[1], lc_bounds[1]),
        min(dem_bounds[2], lc_bounds[2]),
        min(dem_bounds[3], lc_bounds[3])
    )

    # Calculate intersection transform and dimensions
    # both lc_width/height and dem_width/height can be used because it only takes the intersection.
    intersection_transform, intersection_width, intersection_height = calculate_default_transform(
        dst_crs, dst_crs, dem_width, dem_height, *intersection_bounds, resolution=resolution) 
    
    # reprojection
    with rasterio.open(dem_path) as dem_src:
        dem_reprojected = reproject_and_resample(
            dem_src, dst_crs, intersection_transform, intersection_width, intersection_height)

    with rasterio.open(land_cover_path) as lc_src:
        lc_reprojected = reproject_and_resample(
            lc_src, dst_crs, intersection_transform, intersection_width, intersection_height)
        
    return dem_reprojected, lc_reprojected, intersection_transform

def reproject_and_resample(src, dst_crs, dst_transform, dst_width, dst_height, resampling=Resampling.nearest):
    dst_shape = (dst_height, dst_width)
    dst_array = np.empty(dst_shape, dtype=src.read(1).dtype)
    reproject(
        source=src.read(1),
        destination=dst_array,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=resampling
    )
    return dst_array

def reclassify_land_cover_urban_forest(land_cover):    
    # Define the reclassification dictionary from Level II to Level I
    reclass_dict = {
        11: 0, 12: 0, 31: 0, 51: 0, 52: 0, 71: 0, 72: 0, 73: 0, 74: 0, 81: 0, 82: 0, 90: 0, 95: 0, # Others
        21: 2, 22: 2, 23: 2, 24: 2,  # Developed
        41: 4, 42: 4, 43: 4,  # Forest
    }
    reclass_land_cover = np.copy(land_cover)
    for key, value in reclass_dict.items():
        reclass_land_cover[land_cover == key] = value
    return reclass_land_cover

# Calculate slope in degrees 
def calculate_slope(dem, transform):
    x, y = np.gradient(dem, transform[0], transform[4]) # reflect real world distance
    slope = np.arctan(np.sqrt(x**2 + y**2)) * 180 / np.pi
    return slope

# Classify terrain based on slope (USDA Slope Gradient Classification)
def classify_slope(slope):
    classification = np.zeros_like(slope)
    classification[(slope >= 0) & (slope < 2)] = 1  # Flat: <2 degrees
    classification[(slope >= 2) & (slope < 5)] = 2  # Undulating: 2-5 degrees
    classification[(slope >= 5) & (slope < 8)] = 3  # Moderately sloping: 5-8 degrees
    classification[(slope >= 8) & (slope < 17)] = 4  # Hilly: 8-17 degrees
    classification[(slope >= 17) & (slope < 24)] = 5  # Moderately steep: 17-24 degrees
    classification[(slope >= 24) & (slope < 33)] = 6  # Steep: 24-33 degrees
    classification[slope >= 33] = 7  # Very steep: >33 degrees
    return classification

def reclassify_slope(slope):
    reclass_dict = {
        2:1,
        3:2,4:2,
        5:3,6:3,7:3
    }
    reclass_slope = np.copy(slope)
    for key, value in reclass_dict.items():
        reclass_slope[slope == key] = value
    return reclass_slope

def process_maps_class_freq(slope_map, land_cover_map, tile_size, ignore_non_aoi=0.8):
    """
    Args:
    slope_map (2D np.array): Slope classification map
    land_cover_map (2D np.array): Land cover map
    tile_size (int): Size of each tile in pixels (assuming square tiles)
    
    Returns:
    entropy_map (2D np.array): Map of joint entropy values
    """
    # Ensure maps are the same size
    assert slope_map.shape == land_cover_map.shape, "Maps must be the same size"
    
    # Adjust dimensions to be multiples of tile_size
    rows, cols = slope_map.shape
    rows_trimmed = (rows // tile_size) * tile_size
    cols_trimmed = (cols // tile_size) * tile_size
    slope_map = slope_map[:rows_trimmed, :cols_trimmed]
    land_cover_map = land_cover_map[:rows_trimmed, :cols_trimmed]
    
    # Split the maps into tiles
    slope_tiles = view_as_blocks(slope_map, block_shape=(tile_size, tile_size)) # (h, w, tilesize, tilesize)
    land_cover_tiles = view_as_blocks(land_cover_map, block_shape=(tile_size, tile_size))
    h,w, _, _ = land_cover_tiles.shape
    mask = np.sum(land_cover_tiles>0, axis=(-2,-1)) < (tile_size**2 * ignore_non_aoi) # identify where more than XX% pixels are Non AoI class (id: 0)
    
    # get most frequent class as coarse classification map for tile images
    coarse_lc = land_cover_tiles.reshape(h, w, -1)
    coarse_lc = mode(coarse_lc, axis=-1)[0]
    coarse_slope = slope_tiles.reshape(h, w, -1)
    coarse_slope = mode(coarse_slope, axis=-1)[0]
    combined = np.concatenate([coarse_lc[..., np.newaxis], coarse_slope[..., np.newaxis]], -1) # (h,w,2)
    combined_flatten = combined.reshape(-1, 2)
    u, inv, c = np.unique(combined_flatten, axis=0, return_counts=True, return_inverse=True)
    inv_freq = 1 / c
    combined_inv_freq_map = inv_freq[inv].reshape(h, w)
    
    # compute probability map
    combined_inv_freq_map[mask] = 0 # ignore > XX% non AoI region
    if np.sum(combined_inv_freq_map) == 0:
        num_patches_available = 0
        return None, slope_tiles, land_cover_tiles, num_patches_available, mask
    else:
        combined_inv_prob_map = combined_inv_freq_map / np.sum(combined_inv_freq_map) # convert them to probability
        num_patches_available = np.sum(combined_inv_prob_map > 0)
        return combined_inv_prob_map, slope_tiles, land_cover_tiles, num_patches_available, mask


def sample_pixels_by_probability(probability_map, num_samples):
    h, w = probability_map.shape
    n_total_pixels = h * w
    # only performs sampling on non-zero probability tiles 
    mask = (probability_map > 0).reshape(-1)
    n_valid_patches = np.sum(mask)
    flat_index = np.arange(n_total_pixels)
    if num_samples is None or num_samples >= n_valid_patches: 
        # sort the samples with probability. All samples are chosen and ranked.
        sorted_index = np.random.choice(flat_index, n_valid_patches, replace=False, p=probability_map.reshape(-1))
        sampled_indices_2d = np.unravel_index(sorted_index, (h, w)) # take all non-zero prob tiles
        # conver to nsample tuple where each element is an array [index_h, index_w]
        array_h, array_w = sampled_indices_2d
        sampled_indices_2d = [np.array([x, y]) for (x,y) in zip(array_h, array_w) ]
        return sampled_indices_2d
    elif num_samples < n_valid_patches:  
        # sample based on the prob map
        sampled_indices = np.random.choice(flat_index, num_samples, replace=False, p=probability_map.reshape(-1))
        sampled_indices_2d = np.unravel_index(sampled_indices, (h, w)) # tuple of array ((nsample,), (nsample,))
        # conver to nsample tuple where each element is an array [index_h, index_w]
        array_h, array_w = sampled_indices_2d
        sampled_indices_2d = [np.array([x, y]) for (x,y) in zip(array_h, array_w) ]
        return sampled_indices_2d
    elif n_valid_patches == 0:
        return []
    else:
        raise ValueError("Invalid value for num_samples")

def back_trace_tiles(indices, original_tiles, tile_size, transform):
    """
    Back trace the selected pixels to the original tiles and get their geographical bounds.
    
    Args:
    indices (list of tuples): Indices of the selected pixels in the entropy map.
    original_tiles (3D np.array): The original map tiles to trace back to.
    tile_size (int): The size of the tiles in the original map.
    transform (Affine): Affine transform for the map.
    
    Returns:
    traced_tiles (list of 2D np.array): List of tiles from the original map at the specified indices.
    bounds (list of tuples): List of geographical bounds for each tile.
    """
    traced_tiles = [original_tiles[tuple(index)] for index in indices] # extract the chosen tiles by sampled index 
    bounds = [ # get the geographical tile bounds based on sampled index. 
        (
            transform * (index[1] * tile_size, index[0] * tile_size),  # (min_x, min_y)
            transform * ((index[1] + 1) * tile_size, (index[0] + 1) * tile_size)  # (max_x, max_y)
        )
        for index in indices
    ]
    return traced_tiles, bounds

def geo_bounds_to_pixel_indices(bounds, transform):
    """
    Convert geographical bounds to image pixel indices.
    
    Args:
    bounds (tuple): Geographical bounds as (min_x, min_y, max_x, max_y).
    transform (Affine): Affine transform for the map.
    
    Returns:
    tuple: Pixel indices as (min_row, min_col, max_row, max_col).
    """
    min_x, min_y, max_x, max_y = bounds
    
    # Use the inverse of the transform to map coordinates to pixels
    inv_transform = ~transform
    
    # Calculate the pixel coordinates for the min and max bounds
    min_col, min_row = inv_transform * (min_x, min_y)
    max_col, max_row = inv_transform * (max_x, max_y)
    
    # Convert to integer pixel indices
    min_row, min_col = int(np.floor(min_row)), int(np.floor(min_col))
    max_row, max_col = int(np.ceil(max_row)), int(np.ceil(max_col))
    
    return min_row, min_col, max_row, max_col

def get_tile_labels(tile_list):
    tile_labels = [mode(t.ravel()).mode for t in tile_list]
    return tile_labels

def transform_bounds_to_epsg3857(bounds_list, source_epsg):
    # Define the source and target CRS
    source_crs = f"EPSG:{source_epsg}"
    target_crs = "EPSG:3857"
    
    # Create a transformer object
    transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
    
    transformed_bounds_list = []
    
    for bounds in bounds_list:
        # Unpack the bounds
        (minx, miny), (maxx, maxy) = bounds
        
        # Transform the coordinates
        minx_3857, miny_3857 = transformer.transform(minx, miny)
        maxx_3857, maxy_3857 = transformer.transform(maxx, maxy)
        
        # Append the transformed bounds to the list
        transformed_bounds_list.append(((minx_3857, miny_3857), (maxx_3857, maxy_3857)))
    
    return transformed_bounds_list


def to_json_str(lst):
    """
    Converts a nested list containing np.float32 elements to a JSON string.

    Parameters:
    lst (list): A nested list containing np.float32 values.

    Returns:
    str: A JSON string representation of the list.
    """
    # Convert np.float32 elements to standard Python float
    def convert_element(element):
        if isinstance(element, np.float32):
            return int(element)
        elif isinstance(element, list):  # Recursively convert nested lists
            return [convert_element(e) for e in element]
        elif isinstance(element, np.uint8):
            return int(element)
        return element  # Return as-is for other types

    converted_list = convert_element(lst)
    return json.dumps(converted_list)

In [27]:
def reproject_raster_to_match(src_raster, dst_crs):
    """
    Reproject a raster to match the given destination CRS.

    Parameters:
    src_raster (rasterio.DatasetReader): The source raster to be reprojected.
    dst_crs (rasterio.crs.CRS): The destination CRS.

    Returns:
    numpy.ndarray: The reprojected raster data.
    rasterio.Affine: The transform of the reprojected raster.
    """
    transform, width, height = calculate_default_transform(
        src_raster.crs, dst_crs, src_raster.width, src_raster.height, *src_raster.bounds)
    kwargs = src_raster.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with MemoryFile() as memfile:
        with memfile.open(**kwargs) as reprojected_raster:
            for i in range(1, src_raster.count + 1):
                reproject(
                    source=rasterio.band(src_raster, i),
                    destination=rasterio.band(reprojected_raster, i),
                    src_transform=src_raster.transform,
                    src_crs=src_raster.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

            reprojected_data = reprojected_raster.read()
            reprojected_transform = reprojected_raster.transform
    return reprojected_data, reprojected_transform

def load_and_visualize_rasters(raster_path1, raster_path2):
    """
    Loads and visualizes two raster files side by side.

    Parameters:
    raster_path1 (str): The path to the first raster file.
    raster_path2 (str): The path to the second raster file.
    """
    # Open the first raster file (land cover)
    with rasterio.open(raster_path1) as src1:
        raster1 = src1.read(1)
        transform1 = src1.transform
        crs1 = src1.crs

    # Open the second raster file (DEM)
    with rasterio.open(raster_path2) as src2:
        raster2, transform2 = reproject_raster_to_match(src2, crs1)

    # Define a colormap for the land cover map
    # cmap = ListedColormap([
    #     'black',         # outside the polygon
    #     'blue',          # Open Water (11)
    #     'white',         # Perennial Ice/Snow (12)
    #     'lightcoral',    # Developed, open space (21)
    #     'indianred',     # Developed, low intensity (22)
    #     'brown',         # Developed, medium intensity (23)
    #     'darkred',       # Developed, high intensity (24)
    #     'peru',          # Barren land (31)
    #     'forestgreen',   # Deciduous forest (41)
    #     'darkgreen',     # Evergreen forest (42)
    #     'limegreen',     # Mixed forest (43)
    #     'yellowgreen',   # Shrubland (52)
    #     'yellow',        # Grassland (71)
    #     'lightyellow',   # Pasture/hay (81)
    #     'gold',          # Cropland (82)
    #     'saddlebrown',   # Woody wetland (90)
    #     'lightseagreen'  # Herbaceous wetland (95)
    # ])
    cmap = mcolors.ListedColormap([
            'black', # not in AoI
            'blue', 'white',  # Water
            'lightcoral', 'coral', 'tomato', 'orangered',  # Developed
            'gray',  # Barren
            'darkgreen', 'forestgreen', 'limegreen',  # Forest
            'darkkhaki', 'khaki',  # Shrubland
            'lightgoldenrodyellow', 'gold', 'goldenrod', 'darkgoldenrod',  # Herbaceous
            'lightgreen', 'lime',  # Planted/Cultivated
            'lightcyan', 'cyan'  # Wetlands
        ])

    # Define the legend labels
    legend_labels = [
        'outside of AoI', 'Open Water (11)', 'Perennial Ice/Snow (12)', 'Developed, open space (21)',
        'Developed, low intensity (22)', 'Developed, medium intensity (23)', 'Developed, high intensity (24)',
        'Barren land (31)', 'Deciduous forest (41)', 'Evergreen forest (42)',
        'Mixed forest (43)', 'Shrubland (52)', 'Grassland (71)',
        'Pasture/hay (81)', 'Cropland (82)', 'Woody wetland (90)', 'Herbaceous wetland (95)'
    ]

    # Create a patch for each color
    legend_patches = [mpatches.Patch(color=cmap(i), label=label) for i, label in enumerate(legend_labels)]

    # Plot the rasters side by side
    # fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    fig, axes = plt.subplots(2,1, figsize=(15, 10))

    # Plot the first raster (land cover)
    ax1 = axes[0]
    img1 = ax1.imshow(raster1, cmap=cmap, extent=[
        transform1[2], 
        transform1[2] + transform1[0] * raster1.shape[1], 
        transform1[5] + transform1[4] * raster1.shape[0], 
        transform1[5]
    ])
    ax1.set_title('Land Cover Map')
    ax1.legend(handles=legend_patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    print(np.unique(raster1))
    # Plot the second raster (DEM)
    ax2 = axes[1]
    img2 = ax2.imshow(raster2[0], cmap='terrain', extent=[
        transform2[2], 
        transform2[2] + transform2[0] * raster2.shape[2], 
        transform2[5] + transform2[4] * raster2.shape[1], 
        transform2[5]
    ])
    ax2.set_title('Digital Elevation Model')
    plt.colorbar(img2, ax=ax2, orientation='vertical')

    # Show the plot
    plt.tight_layout()
    plt.show()





# sampling

In [3]:
topojson_path = '../../data/point_cloud_boundary/20240627.topojson'
nlcd_all = glob('../../data/NLCD_clip/*.tif')
dem_all = glob('../../data/3DEP_30m_clip/*.tif')
name_list_nlcd_all = ['_'.join(x.split('/')[-1].split('_')[:-2]) for x in nlcd_all]
name_list_dem_all = ['_'.join(x.split('/')[-1].split('_')[:-1]) for x in dem_all]

# filter 
gdf = load_topojson(topojson_path)
gdf_mask = np.array([True if x in name_list_nlcd_all else False for x in gdf.name])
augmented_gdf = gdf[gdf_mask]
print(augmented_gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2054 entries, 37 to 2117
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        2054 non-null   object  
 1   count     2054 non-null   int64   
 2   name      2054 non-null   object  
 3   url       2054 non-null   object  
 4   geometry  2054 non-null   geometry
dtypes: geometry(1), int64(1), object(3)
memory usage: 96.3+ KB
None


## for loop ver

In [8]:
np.random.seed(41) # set seed 
num_samples = 1000  # Number of tiles to sample 

# containers 
names = []
urls = []
local_epsg = []
polygons = []
lc_tile_labels = []
slope_tile_labels = []
skipped_index = []
polygons_local_epsg = []

# loop for each land cover (cropped by point cloud boundary)
for i in range(len(nlcd_all)):
    sample_nlcd = nlcd_all[i]
    sample_dem = dem_all[name_list_dem_all.index(name_list_nlcd_all[i])]
    
    # get the EPSG code for this crop
    region_data = augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]]
    epsg_code, area_utm, (lat_center, lon_center) = reproject_to_utm_and_calculate_area(region_data)
    density_local = augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]]['count'].values[0] / area_utm
    dst_crs = f'EPSG:{epsg_code}'
    print(f'{i+1}/{len(nlcd_all)} {name_list_nlcd_all[i]}. {dst_crs}: area is {area_utm/1e6:.2f} km2, density is {density_local:.2f} p/m2')
    
    # File Paths to DEM and land cover data
    dem_path = sample_dem
    land_cover_path = sample_nlcd
    
    # Align rasters
    align_resolution = 25 # the resolution when they are aligned. CRS units.
    dem, land_cover, intersection_transform = align_rasters(dem_path, land_cover_path, dst_crs, resolution=align_resolution)
    pixel_length = np.abs(intersection_transform[0])
    assert pixel_length == align_resolution
    assert dem.shape == land_cover.shape
    
    # land cover reclassification from levle II to level I
    reclassified_land_cover = reclassify_land_cover_urban_forest(land_cover)
    
    # mask out outside of AoI region of Dem with Land cover labels (artifacts due to reprojection)
    mask = reclassified_land_cover == 0
    dem[mask] = np.nan
    
    # Calculate the slope from DEM and classify into USDA classes
    slope = calculate_slope(dem, intersection_transform)
    classified_slope = classify_slope(slope)
    classified_slope = reclassify_slope(classified_slope)
    
    # compute inverse frequnecy probability map for tile sampling 
    slope_map = classified_slope
    land_cover_map = reclassified_land_cover
    tile_size = int(500 // pixel_length)
    
    prob_map, slope_tiles, land_cover_tiles, num_patches_available, mask = process_maps_class_freq(slope_map, land_cover_map, tile_size)

    if num_patches_available == 0:
        print(f'Skip {i}th sample due to zero tiles')
        skipped_index.append(i)
        continue
    
    # Sample pixels (tiles) from the probability map
    geotransform = intersection_transform # pixel space to geographical transform 
    sampled_indices = sample_pixels_by_probability(prob_map, num_samples)
    if sampled_indices == 0:
        print(f'Skip {i}th sample due to zero tiles')
        skipped_index.append(i)
        continue
    
    # Back trace the selected pixels to the original tiles and get their bounds
    sampled_slope_tiles, sampled_bounds = back_trace_tiles(sampled_indices, slope_tiles, tile_size, geotransform)
    sampled_land_cover_tiles, _ = back_trace_tiles(sampled_indices, land_cover_tiles, tile_size, geotransform)
    sampled_pixel_indices = [geo_bounds_to_pixel_indices([item for subtuple in bounds for item in subtuple], geotransform) for bounds in sampled_bounds]
    sampled_land_cover_tile_labels = get_tile_labels(sampled_land_cover_tiles)
    sampled_slope_tile_labels = get_tile_labels(sampled_slope_tiles)
    
    # obtain geo bounds in EPSG 3857 for data extraction in 3DEP database 
    sampled_bounds_epsg3857 = transform_bounds_to_epsg3857(sampled_bounds, epsg_code)
    
    names.append(name_list_nlcd_all[i])
    urls.append(augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]].url.values[0])
    local_epsg.append(epsg_code)
    polygons.append(sampled_bounds_epsg3857)
    lc_tile_labels.append(sampled_land_cover_tile_labels)
    slope_tile_labels.append(sampled_slope_tile_labels)
    polygons_local_epsg.append(sampled_bounds)
    
    # break 

1/2054 USGS_LPC_MN_Phase1_RedwoodCO_2010_LAS_2016. EPSG:32615: area is 2780.68 km2, density is 1.26 p/m2
2/2054 MN_FullState. EPSG:32615: area is 244710.47 km2, density is 1.19 p/m2



KeyboardInterrupt



## parallel ver.

In [5]:
# Define the function to be run in parallel
def process_sample(i):
    sample_nlcd = nlcd_all[i]
    sample_dem = dem_all[name_list_dem_all.index(name_list_nlcd_all[i])]
    
    # get the EPSG code for this region 
    region_data = augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]]
    epsg_code, area_utm, (lat_center, lon_center) = reproject_to_utm_and_calculate_area(region_data)
    density_local = augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]]['count'].values[0] / area_utm
    dst_crs = f'EPSG:{epsg_code}'
    print(f'{i+1}/{len(nlcd_all)} {name_list_nlcd_all[i]}. {dst_crs}: area is {area_utm/1e6:.2f} km2, density is {density_local:.2f} p/m2')
    
    # Paths to DEM and land cover data
    dem_path = sample_dem
    land_cover_path = sample_nlcd
    
    # Align rasters
    align_resolution = 25 # the resolution when they are aligned. CRS units
    dem, land_cover, intersection_transform = align_rasters(dem_path, land_cover_path, dst_crs, resolution=align_resolution)
    pixel_length = np.abs(intersection_transform[0])
    assert pixel_length == align_resolution
    assert dem.shape == land_cover.shape
    
    # land cover reclassification from levle II to level I
    reclassified_land_cover = reclassify_land_cover_urban_forest(land_cover)
    
    # mask out outside of AoI region of Dem with Land cover labels. (artifacts due to reprojection)
    mask = reclassified_land_cover == 0
    dem[mask] = np.nan
    
    # Calculate the slope from DEM and classify into USDA classes
    slope = calculate_slope(dem, intersection_transform)
    classified_slope = classify_slope(slope)
    classified_slope = reclassify_slope(classified_slope)
    
    # compute inverse frequnecy probability map for tile sampling 
    slope_map = classified_slope
    land_cover_map = reclassified_land_cover
    tile_size = int(500 // pixel_length)  # 50x50 pixels for 500m x 500m tiles if each pixel is 10m x 10m
    
    prob_map, slope_tiles, land_cover_tiles, num_patches_available, mask = process_maps_class_freq(slope_map, land_cover_map, tile_size)

    if num_patches_available == 0:
        print(f'Skip {i}th sample due to zero tiles')
        return i, None
    # Sample pixels (tiles) from the probability map
    geotransform = intersection_transform # pixel space to geographical transform 
    sampled_indices = sample_pixels_by_probability(prob_map, num_samples)
    if sampled_indices == 0:
        print(f'Skip {i}th sample due to zero tiles')
        return i, None
    
    # Back trace the selected pixels to the original tiles and get their bounds
    sampled_slope_tiles, sampled_bounds = back_trace_tiles(sampled_indices, slope_tiles, tile_size, geotransform)
    sampled_land_cover_tiles, _ = back_trace_tiles(sampled_indices, land_cover_tiles, tile_size, geotransform)
    sampled_pixel_indices = [geo_bounds_to_pixel_indices([item for subtuple in bounds for item in subtuple], geotransform) for bounds in sampled_bounds]
    sampled_land_cover_tile_labels = get_tile_labels(sampled_land_cover_tiles)
    sampled_slope_tile_labels = get_tile_labels(sampled_slope_tiles)
    
    # obtain geo bounds in EPSG 3857 for data extraction in 3DEP database 
    sampled_bounds_epsg3857 = transform_bounds_to_epsg3857(sampled_bounds, epsg_code)
    
    return i, {
        'name': name_list_nlcd_all[i],
        'url': augmented_gdf[augmented_gdf.name == name_list_nlcd_all[i]].url.values[0],
        'epsg_code': epsg_code,
        'bounds': sampled_bounds_epsg3857,
        'lc_labels': sampled_land_cover_tile_labels,
        'slope_labels': sampled_slope_tile_labels,
        'local_bounds': sampled_bounds
    }

# Create lists to store results
names = []
urls = []
local_epsg = []
polygons = []
lc_tile_labels = []
slope_tile_labels = []
skipped_index = []
polygons_local_epsg = []

# Set the number of CPUs
num_cpus = 4  # Adjust this to the number of CPUs you want to use

# Use joblib.Parallel to parallelize the for loop
results = Parallel(n_jobs=num_cpus, timeout=300)(delayed(process_sample)(i) for i in range(len(nlcd_all)))

# Collect results
for i, result in results:
    if result is None:
        skipped_index.append(i)
    else:
        names.append(result['name'])
        urls.append(result['url'])
        local_epsg.append(result['epsg_code'])
        polygons.append(result['bounds'])
        lc_tile_labels.append(result['lc_labels'])
        slope_tile_labels.append(result['slope_labels'])
        polygons_local_epsg.append(result['local_bounds'])

1/2054 USGS_LPC_MN_Phase1_RedwoodCO_2010_LAS_2016. EPSG:32615: area is 2780.68 km2, density is 1.26 p/m2
5/2054 USGS_LPC_IL_12_County_Stephenson_Co_2009_LAS_2016. EPSG:32616: area is 1650.69 km2, density is 2.06 p/m2
6/2054 WY_SouthCentral_5_2020. EPSG:32613: area is 4520.55 km2, density is 59.67 p/m2


/Users/hirokishuu/miniconda3/envs/geospatial/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(

KeyboardInterrupt



## save the sampled tiles 

In [9]:
num_samples = 1000
slope_tile_labels_json = [to_json_str(region_labels) for region_labels in slope_tile_labels]
lc_tile_labels_json = [to_json_str(region_labels) for region_labels in lc_tile_labels]

# Create a list of polygons from bounds
multi_polygon_list = []
for region_polygons in polygons: 
    # region polygons: length 100 ((minx, miny), (maxx, maxy))
    polygon_list = [Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)]) for ((minx, miny), (maxx, maxy)) in region_polygons]
    multi_polygon = MultiPolygon(polygon_list)
    multi_polygon_list.append(multi_polygon)
# Create a dictionary with the data
data = {
    'name': names,
    'url': urls,
    'local_epsg_code': local_epsg,
    'slope_tile_labels': slope_tile_labels_json,
    'land_cover_tile_labels': lc_tile_labels_json, # json string reading is much faster for gpkg compareed to int or float
    'bounds_in_local_epsg': polygons_local_epsg,
    'geometry': multi_polygon_list
}

# Create the GeoDataFrame
gdf = gpd.GeoDataFrame(data, crs="EPSG:3857")
gdf.to_file(f"../../data/sampled_tiles/sample_{num_samples}_developed_forest.gpkg", driver="GPKG", engine='pyogrio')